In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from pipelines.dig_a_plan_slack_v import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import pandapower_to_dig_a_plan_schema

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
LOAD_FACTOR  = 1

net = pp.from_pickle(".cache/input_data/mv_example.p")

net["load"]["p_mw"] = net["load"]["p_mw"]*LOAD_FACTOR
net["load"]["q_mvar"] = net["load"]["q_mvar"]*LOAD_FACTOR
net["trafo"]["vn_hv_kv"] = [105., 110.]
net["trafo"]["vn_lv_kv"] = [20., 19.]

line_list= [6, 9, 11, 13]

# net["line"].loc[line_list, "max_i_ka"] = 1e-3

In [3]:
grid_data = pandapower_to_dig_a_plan_schema(net=net)

dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e4, penalty_cost = 1e1
)

dig_a_plan.add_grid_data(**grid_data)
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: 0, Slave obj: 0 and Gap: 1e6:   0%|          | 0/200 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [ ]:
compare_dig_a_plan_with_pandapower(dig_a_plan=dig_a_plan, net=net)
plot_grid_from_pandapower(net=net)

TypeError: unsupported format string passed to NoneType.__format__

In [ ]:
dig_a_plan.extract_edge_current()

edge_id,i_pu,eq_fk,i_base
i64,f64,str,f64
0,0.0,"""Line 1-2""",28.867513
1,0.0,"""Line 2-3""",28.867513
2,0.0,"""Line 3-4""",28.867513
3,0.0,"""Line 4-5""",28.867513
4,0.0,"""Line 5-6""",28.867513
…,…,…,…
14,0.0,"""Line 14-8""",28.867513
15,0.0,"""Line 3-10""",28.867513
16,0.0,"""Line 11-13""",28.867513


In [ ]:
dig_a_plan.slave_model_instance.master_d.pprint()

master_d : Size=72, Index=LC, Domain=Any, Default=0, Mutable=True
    Key          : Value
      (0, 1, 15) :   0.0
      (0, 15, 1) :   1.0
      (1, 2, 16) :   0.0
      (1, 16, 2) :   1.0
      (2, 3, 17) :   0.0
      (2, 17, 3) :   1.0
      (3, 4, 18) :   0.0
      (3, 18, 4) :   1.0
      (4, 5, 19) :   0.0
      (4, 19, 5) :   1.0
      (5, 7, 20) :   0.0
      (5, 20, 7) :   1.0
      (6, 8, 21) :   0.0
      (6, 21, 8) :   1.0
      (7, 9, 22) :   1.0
      (7, 22, 9) :   0.0
     (8, 10, 23) :   1.0
     (8, 23, 10) :   0.0
      (9, 3, 24) :   0.0
      (9, 24, 3) :   1.0
    (10, 12, 25) :   0.0
    (10, 25, 12) :   1.0
    (11, 13, 26) :   0.0
    (11, 26, 13) :   1.0
     (12, 6, 27) :   0.0
     (12, 27, 6) :   1.0
    (13, 11, 28) :   0.0
    (13, 28, 11) :   1.0
    (14, 14, 29) :   0.0
    (14, 29, 14) :   1.0
     (15, 6, 30) :   0.0
     (15, 30, 6) :   1.0
    (16, 11, 31) :   1.0
    (16, 31, 11) :   0.0
      (17, 0, 1) :   0.0
      (17, 1, 0) :   1.0
     (18,

In [ ]:
dig_a_plan.slave_model_instance.current_rotated_cone.pprint()

current_rotated_cone : Size=72, Index=LC, Active=True
    Key          : Lower : Body           : Upper : Active
      (0, 1, 15) :   0.0 :   i_sq[0,1,15] :   0.0 :   True
      (0, 15, 1) :   0.0 :   i_sq[0,15,1] :   0.0 :   True
      (1, 2, 16) :   0.0 :   i_sq[1,2,16] :   0.0 :   True
      (1, 16, 2) :   0.0 :   i_sq[1,16,2] :   0.0 :   True
      (2, 3, 17) :   0.0 :   i_sq[2,3,17] :   0.0 :   True
      (2, 17, 3) :   0.0 :   i_sq[2,17,3] :   0.0 :   True
      (3, 4, 18) :   0.0 :   i_sq[3,4,18] :   0.0 :   True
      (3, 18, 4) :   0.0 :   i_sq[3,18,4] :   0.0 :   True
      (4, 5, 19) :   0.0 :   i_sq[4,5,19] :   0.0 :   True
      (4, 19, 5) :   0.0 :   i_sq[4,19,5] :   0.0 :   True
      (5, 7, 20) :   0.0 :   i_sq[5,7,20] :   0.0 :   True
      (5, 20, 7) :   0.0 :   i_sq[5,20,7] :   0.0 :   True
      (6, 8, 21) :   0.0 :   i_sq[6,8,21] :   0.0 :   True
      (6, 21, 8) :   0.0 :   i_sq[6,21,8] :   0.0 :   True
      (7, 9, 22) :   0.0 :   i_sq[7,9,22] :   0.0 :   True
  